In [ ]:
import polars as pl

In [ ]:
data = pl.read_csv('./data/ramirez2022/RAMIREZ2022.csv', has_header=True, separator=';')

In [ ]:
from urllib.request import urlopen, HTTPError
from urllib.parse import quote
from time import sleep


def to_InChI(ids: str) -> str:
    try:
        url = (
            "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/"
            + quote(ids)
            + "/property/InChI/TXT"
        )
        with urlopen(url) as ans:
            ans = ans.read().decode("utf8").rstrip()
            ans = ans.split("\n")[0]

    except (TypeError, ValueError, HTTPError):
        try:
            url = (
                "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/"
                + quote(ids)
                + "/property/InChI/TXT"
            )
            ans = urlopen(url).read().decode("utf8").rstrip()
            ans = ans.split("\n")[0]
        except (TypeError, ValueError, HTTPError):
            print("not ok:", url)
            ans = None

    sleep(0.1)
    return ans

In [ ]:
inchis = data['name'].apply(to_InChI).rename('inchi')

In [ ]:
inchis.describe()

In [ ]:
data = data.with_columns(inchis)

In [ ]:
data.write_parquet("./data/ramirez2022/raw/data.parquet")